In [2]:
import pandas as pd
import requests
import boto3


from database_utils import DatabaseConnector 
from database_utils import DataExtractor
from database_utils import DataCleaning

database_connector = DatabaseConnector()
data_extractor = DataExtractor()
data_cleaning = DataCleaning()

data_extractor = DataExtractor()
my_bucket = data_extractor.extract_from_s3('s3://data-handling-public/products.csv')
print(my_bucket)


      Unnamed: 0                                 product_name product_price   
0              0  FurReal Dazzlin' Dimples My Playful Dolphin        £39.99  \
1              1          Tiffany's World Day Out At The Park        £12.99   
2              2          Tiffany's World Pups Picnic Playset         £7.00   
3              3     Tiffany's World Wildlife Park Adventures        £12.99   
4              4                      Cosatto Cosy Dolls Pram        £30.00   
...          ...                                          ...           ...   
1848        1848    Goodmans Qi Autosense In Car Phone Holder        £15.00   
1849        1849    Goodmans Qi Wireless Charging Phone Stand        £15.00   
1850        1850                     RAC 12V Wet & Dry Vacuum        £18.00   
1851        1851    Goodmans Qi Autosense In Car Phone Holder        £15.00   
1852        1852               Beldray Telescopic Ladder 2.9m        £69.00   

      weight        category            EAN  date_a

In [3]:
my_bucket.describe()

,Unnamed: 0
count,1853.000000
mean,926.000000
std,535.059343
min,0.000000
25%,463.000000
50%,926.000000
75%,1389.000000
max,1852.000000


In [12]:
my_bucket.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1853 entries, 0 to 1852
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     1853 non-null   int64 
 1   product_name   1849 non-null   object
 2   product_price  1849 non-null   object
 3   weight         1583 non-null   string
 4   category       1849 non-null   object
 5   EAN            1849 non-null   object
 6   date_added     1849 non-null   object
 7   uuid           1849 non-null   object
 8   removed        1849 non-null   object
 9   product_code   1849 non-null   object
dtypes: int64(1), object(8), string(1)
memory usage: 144.9+ KB


In [18]:
my_bucket['weight'] = my_bucket['weight'].astype('float64')

ValueError: could not convert string to float: '1.16kg'

In [19]:
  
import re 
# Define a regular expression pattern to match weight values
weight_pattern = r'^([\d\.]+)\s*(kg|g|ml|l|oz|lb)$'
        
# Define a conversion factor for ml to g
ml_to_g_factor = 1
     
# Define a dictionary mapping weight units to conversion factors
conversion_factors = {
            'kg': 1,
            'g': 0.001,
            'ml': ml_to_g_factor,
            'l': 1000,
            'oz': 0.0283495,
            'lb': 0.453592
        }
        
# Clean up the weight column by removing excess characters
my_bucket['weight'] = my_bucket['weight'].str.replace('[^0-9\.kgml]', 'kg', regex=True)
        
# Convert each weight value to a decimal value in kg
for i, weight in enumerate(my_bucket['weight']):
    match = re.match(weight_pattern, weight)
    if match:
        value, unit = match.groups()
        factor = conversion_factors[unit]
        my_bucket.at[i, 'weight'] = float(value) * factor
    else:
        my_bucket.at[i, 'weight'] = None
        
        
     

TypeError: expected string or bytes-like object, got 'NAType'